In [ ]:
'''
#필요없음
#test data
file_number = 200000
for i in range(20874):
    with open('data/test/test_' + str(file_number + i) + '.json') as json_file:
        json_data = json.load(json_file)
        
        #파일 이름
        json_file_name = json_data["file_name"]
        file_name_string = json_file_name.split('.json')
        #print(strings)
        
        #imageData
        json_imageData = json_data["imageData"]
        #print(json_imageData)

    imgdata = base64.b64decode(json_imageData)
    savepath = 'test_image/'
    
    filename = savepath + file_name_string[0] + '.jpg'  
    with open(filename, 'wb') as f:
        f.write(imgdata)
'''

In [ ]:
# json 모듈을 이용하여 데이터 파싱
import json
# shape 안의 label과 point를 따로 저장
import csv
#디렉토리 없으면 만들기
import os

def makedirs(path): 
   try: 
        os.makedirs(path) 
   except OSError: 
       if not os.path.isdir(path): 
           raise

# Base64 코드를 이미지로 디코딩
#https://stackoverflow.com/questions/16214190/how-to-convert-base64-string-to-image/16214280
import base64


#train data
file_number = 100000
for i in range(62622):
    with open('data/train/train_' + str(file_number + i) + '.json') as json_file:
        json_data = json.load(json_file)
        
        #파일 이름
        json_file_name = json_data["file_name"]
        file_name_string = json_file_name.split('.json')
        #print(strings)
        
        #shapes
        json_shapes = json_data["shapes"]
        #print(json_shapes)
        
        #imageData
        json_imageData = json_data["imageData"]
        #print(json_imageData)

    imgdata = base64.b64decode(json_imageData)
    #10%는 test 데이터로 추출
    savepath = 'train_image/' if i < 56358  else 'test_image_from_train_image/'
    labelname = json_shapes[0]['label']
    dirname = savepath + labelname
    makedirs(dirname)
    
    filename = savepath + labelname + '/' + file_name_string[0] + '.jpg'  
    with open(filename, 'wb') as f:
        f.write(imgdata)

In [1]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf 

from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

from tensorflow.keras.applications import efficientnet
from tensorflow.keras.applications import EfficientNetB2

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

preprocessor = efficientnet.preprocess_input

conv_layers = EfficientNetB2(weights='imagenet', include_top=False, input_shape=(576, 576, 3))
conv_layers.trainable = False

model = keras.Sequential()

model.add(conv_layers)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(4, activation='softmax')) #4가지로 나뉘니까 출력층을 4개로 변경

model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSprop", metrics=['acc'])
model.summary()

BATCH_SIZE = 32

train_data_generator = ImageDataGenerator(
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      vertical_flip=False,
      preprocessing_function=preprocessor
).flow_from_directory(
      "train_image",
      target_size=(576,576),
      batch_size=BATCH_SIZE,
      class_mode='sparse'
)

test_data_generator = ImageDataGenerator(
      preprocessing_function=preprocessor
).flow_from_directory(
      "test_image_from_train_image",
      target_size=(576,576),
      batch_size=BATCH_SIZE,
      class_mode='sparse'
)


history = model.fit(
      train_data_generator,
      validation_data=test_data_generator,
      epochs=10
)


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()


loss, acc = model.evaluate(test_data_generator)
print("loss=", loss)
print("acc=", acc)



batch_x, batch_y = test_data_generator.next()
y_ = model.predict(batch_x)
predicted = np.argmax(y_, axis=-1)

plt.plot(batch_y[:100], "o")
plt.plot(predicted[:100], '.')
plt.show()


custom_labels = list(test_data_generator.class_indices.keys())
print("label category index =", batch_y[0])
print("predicted category index =", predicted[0])
print("predicted category name =", custom_labels[predicted[0]])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb2 (Functional)  (None, 18, 18, 1408)     7768569   
                                                                 
 flatten (Flatten)           (None, 456192)            0         
                                                                 
 dense (Dense)               (None, 256)               116785408 
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 124,556,029
Trainable params: 116,786,948
Non-trainable params: 7,769,081
__________________________________

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/efficientnetb2/stem_conv/Conv2D
 (defined at /opt/conda/lib/python3.7/site-packages/keras/layers/convolutional.py:238)
]] [Op:__inference_train_function_20458]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/efficientnetb2/stem_conv/Conv2D:
In[0] sequential/efficientnetb2/stem_conv_pad/Pad (defined at /opt/conda/lib/python3.7/site-packages/keras/backend.py:3672)	
In[1] sequential/efficientnetb2/stem_conv/Conv2D/ReadVariableOp:

Operation defined at: (most recent call last)
>>>   File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
>>>     "__main__", mod_spec)
>>> 
>>>   File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/opt/conda/lib/python3.7/site-packages